In [ ]:
# imports
import os
import shutil
from pathlib import Path
import urllib.request
import pandas as pd
from google.cloud import bigquery
from prefect import task, flow
from prefect_gcp.cloud_storage import GcsBucket

print("Setup Complete")


# Deployment 1
# Fetch the data from Github url
@task(log_prints=True, name="fetch-file-dataset_url")
def fetch(dataset_url: str):
    filename, _ = urllib.request.urlretrieve(dataset_url)
    return filename


# Read and tweak to fix the dtypes of pick-up and drop-off
@task(log_prints=True, name="read-and-tweak-df")
def read_tweak_df(src: str) -> pd.DataFrame:
    df = pd.read_csv(src, parse_dates=[2, 3], compression="gzip")
    return df


# Write DataFrame to a specific folder after tweaking the DataFrame
@task(log_prints=True, name="write-to-local-file")
def write_local(df: pd.DataFrame, year: int, dataset_file: str) -> Path:
    directory = Path(f"{year}")
    path_name = directory / f"{dataset_file}.csv.gz"
    try:
        os.makedirs(directory, exist_ok=True)
        df.to_csv(path_name, compression="gzip")
    except OSError as error:
        print(error)
    return path_name


# https://app.prefect.cloud/account/975bd9ed-5aef-4c8a-a413-23073fef3acb/workspace/a711abba-f5ae-4b00-9315-34f92f089b77/blocks/catalog
# Upload local csv.gz file to GCS
@task(log_prints=True)
def write_gcs(path: Path) -> None:
    gcs_block = GcsBucket.load("prefect-gcs-block-ny-taxi")
    gcs_block.upload_from_path(from_path=path, to_path=path)
    print("Loaded data to GCS...Hooray!")
    return


# Delete file after uploads
@task(log_prints=True, name="dedeplicate-local-data")
def deduplicate(path: Path) -> None:
    try:
        os.remove(path)
        os.rmdir(path, )
        print("Successfully deleted directory and local files...hep hep hooray")
    except OSError as error:
        print(f"The system cannot find the file specified: {error}")
    return


@flow(log_prints=True, name="etl-web-gcs")
def etl_web_gcs(year: int, month: int):
    # Parameters
    year = 2019
    month = 5
    dataset_file = f"fhv_tripdata_{year}-{month:02}"
    dataset_url = f"https://github.com/DataTalksClub/nyc-tlc-data/releases/download/fhv/{dataset_file}.csv.gz"

    # Execution
    # Fetch the data from Github url
    source = fetch(dataset_url)

    # no choice but to tweak the data types for us to upload this to BQ
    # Read and tweak to fix the dytpes of pick-up and drop-off
    df_tweak = read_tweak_df(source)

    # write df to local
    path_local = write_local(df_tweak, year, dataset_file)

    # Upload dataset from local to gcs
    write_gcs(path_local)

    # Delete file after uploads
    deduplicate(path_local)

    # Next step is to create a separate deployment from GCS to BigQuery


# Create Parent flow to loop
@flow(name="etl-parent-local-gcs")
def etl_parent_web_gcs(year: int, months: list):
    year = 2019
    months = [2]

    for month in months:
        etl_web_gcs(year, month)


# Run main
if __name__ == "__main__":
    etl_parent_web_gcs(year=2019, months=[2])


In [11]:
# import
from prefect_gcp import GcpCredentials
from google.cloud import bigquery

# Deployment 2



# Upload data from GCS to BigQuery
@flow(log_prints=True, name="etl-gcs-to-bq")
def etl_gcs_to_bq(year: int, month: int):
    
    gcp_creds_block = GcpCredentials.load("prefect-gcs-2023-creds")
    gcp_creds = gcp_creds_block.get_credentials_from_service_account()
    client = bigquery.Client(credentials=gcp_creds)
    table_id = "dtc-de-2023.ny_taxi.ny_taxi_tripdata_2019"

    job_config = bigquery.LoadJobConfig(
        source_format=bigquery.SourceFormat.CSV,
        schema=[
            bigquery.SchemaField("dispatching_base_num", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("pickup_datetime", "TIMESTAMP", mode="NULLABLE"),
            bigquery.SchemaField("dropOff_datetime", "TIMESTAMP", mode="NULLABLE"),
            bigquery.SchemaField("PUlocationID", "FLOAT", mode="NULLABLE"),
            bigquery.SchemaField("DOlocationID", "FLOAT", mode="NULLABLE"),
            bigquery.SchemaField(
                "SR_Flag",
                "FLOAT",
                mode="NULLABLE",
            ),
            bigquery.SchemaField("Affiliated_base_number", "STRING", mode="NULLABLE"),
        ],
    )
    uri = f"gs://ny_taxi_bucket_de_2023/2019/fhv_tripdata_{year}-{month:02}.csv.gz"

    load_job = client.load_table_from_uri(
        uri, table_id, job_config=job_config
    )  # Make an API request.

    load_job.result()  # Waits for the job to complete.

    destination_table = client.get_table(table_id)
    print(f"Loaded {destination_table.num_rows} rows.")


# Parent flow ETL
@flow(log_prints=True, name="etl-parent-to-bq")
def etl_parent_bq_flow(
    year: int = 2019, months: list[int] = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
):
    for month in months:
        etl_gcs_to_bq(year, month)


# run main
if __name__ == "__main__":
    year = 2019
    months = [2]

    etl_parent_bq_flow(year, months)

/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/prefect/tasks.py:261: UserWarning: A task named 'get-gcp-creds' and defined at '/var/folders/h2/6110hgtx6b7400ybw7yjhn400000gn/T/ipykernel_21033/4186519071.py:7' conflicts with another task. Consider specifying a unique `name` parameter in the task definition:

 `@task(name='my_unique_name', ...)`
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/prefect/flows.py:206: UserWarning: A flow named 'etl-gcs-to-bq' and defined at '/var/folders/h2/6110hgtx6b7400ybw7yjhn400000gn/T/ipykernel_21033/4186519071.py:15' conflicts with another flow. Consider specifying a unique `name` parameter in the flow definition:

 `@flow(name='my_unique_name', ...)`
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/prefect/flows.py:206: UserWarning: A flow named 'etl-parent-to-bq' and defined at '/var/folders/h2/6110hgtx6b7400ybw7yjhn400000gn/T/ipykernel_21033/4186519071.py:51' con

15:08:02.452 | INFO    | prefect.engine - Created flow run 'hasty-wildcat' for flow 'etl-parent-to-bq'

/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/prefect/tasks.py:261: UserWarning: A task named 'etl-gcs-to-bq' and defined at '/var/folders/h2/6110hgtx6b7400ybw7yjhn400000gn/T/ipykernel_21033/4186519071.py:15' conflicts with another task. Consider specifying a unique `name` parameter in the task definition:

 `@task(name='my_unique_name', ...)`
  warnings.warn(


15:08:05.498 | INFO    | Flow run 'hasty-wildcat' - Created subflow run 'urban-pig' for flow 'etl-gcs-to-bq'

15:08:06.600 | INFO    | Flow run 'urban-pig' - Created task run 'get-gcp-creds-b52b4cc6-0' for task 'get-gcp-creds'

...

15:08:10.384 | ERROR   | Flow run 'urban-pig' - Encountered exception during execution:

15:08:10.825 | ERROR   | Flow run 'urban-pig' - Finished in state Failed('Flow run encountered an exception. google.api_core.exceptions.BadRequest: 400 Error while reading data, error message: Too many values in row starting at position: 0. Found 8 column(s) while expected 7. File: gs://ny_taxi_bucket_de_2023/2019/fhv_tripdata_2019-02.csv.gz\n')

15:08:10.829 | ERROR   | Flow run 'hasty-wildcat' - Encountered exception during execution:
Traceback (most recent call last):
  File "/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/prefect/engine.py", line 643, in orchestrate_flow_run
    result = await run_sync(flow_call)
  File "/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/prefect/utilities/asyncutils.py", line 156, in run_sync_in_interruptible_worker_thread
    tg.start_soon(
  File "/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/anyio/_backends/_asyncio.py", line 662, in __aexit__
    raise exceptions[0]
  File "/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/anyio/to_thread.py", line 31, in run_sync
    return await get_asynclib().run_sync_in_worker_thread(
  File "/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/anyio/_backends/_asyncio.py", line 937, in run_sync_in_worker_thread
    return await future
  File "/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/anyio/_backends/_asyncio.py", line 867, in run
    result = context.run(func, *args)
  File "/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/prefect/utilities/asyncutils.py", line 135, in capture_worker_thread_and_result
    result = __fn(*args, **kwargs)
  File "/var/folders/h2/6110hgtx6b7400ybw7yjhn400000gn/T/ipykernel_21033/4186519071.py", line 56, in etl_parent_bq_flow
    etl_gcs_to_bq(year, month)
  File "/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/prefect/flows.py", line 448, in __call__
    return enter_flow_run_engine_from_flow_call(
  File "/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/prefect/engine.py", line 175, in enter_flow_run_engine_from_flow_call
    return run_async_from_worker_thread(begin_run)
  File "/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/prefect/utilities/asyncutils.py", line 177, in run_async_from_worker_thread
    return anyio.from_thread.run(call)
  File "/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/anyio/from_thread.py", line 49, in run
    return asynclib.run_async_from_thread(func, *args)
  File "/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/anyio/_backends/_asyncio.py", line 970, in run_async_from_thread
    return f.result()
  File "/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/concurrent/futures/_base.py", line 446, in result
    return self.__get_result()
  File "/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/concurrent/futures/_base.py", line 391, in __get_result
    raise self._exception
  File "/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/prefect/client/utilities.py", line 47, in with_injected_client
    return await fn(*args, **kwargs)
  File "/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/prefect/engine.py", line 553, in create_and_begin_subflow_run
    return await terminal_state.result(fetch=True)
  File "/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/prefect/states.py", line 89, in _get_state_result
    raise await get_state_exception(state)
  File "/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/prefect/engine.py", line 643, in orchestrate_flow_run
    result = await run_sync(flow_call)
  File "/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/prefect/utilities/asyncutils.py", line 156, in run_sync_in_interruptible_worker_thread
    tg.start_soon(
  File "/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/anyio/_backends/_asyncio.py", line 662, in __aexit__
    raise exceptions[0]
  File "/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/anyio/to_thread.py", line 31, in run_sync
    return await get_asynclib().run_sync_in_worker_thread(
  File "/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/anyio/_backends/_asyncio.py", line 937, in run_sync_in_worker_thread
    return await future
  File

15:08:11.188 | ERROR   | Flow run 'hasty-wildcat' - Finished in state Failed('Flow run encountered an exception. google.api_core.exceptions.BadRequest: 400 Error while reading data, error message: Too many values in row starting at position: 0. Found 8 column(s) while expected 7. File: gs://ny_taxi_bucket_de_2023/2019/fhv_tripdata_2019-02.csv.gz\n')

BadRequest: 400 Error while reading data, error message: Too many values in row starting at position: 0. Found 8 column(s) while expected 7. File: gs://ny_taxi_bucket_de_2023/2019/fhv_tripdata_2019-02.csv.gz